In [2]:
import requests

In [3]:
auth = '1hMEqSwhQWWRo88SupMVzQ==', '6b2ffab0-5916-4a20-a0d7-8f9824627d7b'

In [4]:
from httplib import HTTPConnection
import json

known_services = [
'BusStops'
]

credentials = {
    "api_token": None,
    "uuid": None
}

def authenticate(api_token, uuid):
    credentials['api_token'] = api_token
    credentials['uuid'] = uuid

def get(service, start = 0, end=None):
    conn = HTTPConnection('datamall2.mytransport.sg')

    return_value = []

    while end == None or start < end:
        print  '/ltaodataservice/%s?$skip=%i' % (service, start)
        conn.request('GET', '/ltaodataservice/%s?$skip=%i' % (service, start),
            headers = {
                'AccountKey': credentials['api_token'],
                'UniqueUserId': credentials['uuid'],
                'Accept': 'application/json'
            })

        resp = conn.getresponse()
        headers = resp.getheaders()
        body = resp.read()

        data = json.loads(body)

        return_value += data['value']

        if len(data['value']) == 0:
            break

        start += 50

    return return_value

In [5]:
authenticate(*auth)

In [12]:
BusRoutes = get('BusRoutes')

/ltaodataservice/BusRoutes?$skip=0
/ltaodataservice/BusRoutes?$skip=50
/ltaodataservice/BusRoutes?$skip=100
/ltaodataservice/BusRoutes?$skip=150
/ltaodataservice/BusRoutes?$skip=200
/ltaodataservice/BusRoutes?$skip=250
/ltaodataservice/BusRoutes?$skip=300
/ltaodataservice/BusRoutes?$skip=350
/ltaodataservice/BusRoutes?$skip=400
/ltaodataservice/BusRoutes?$skip=450
/ltaodataservice/BusRoutes?$skip=500
/ltaodataservice/BusRoutes?$skip=550
/ltaodataservice/BusRoutes?$skip=600
/ltaodataservice/BusRoutes?$skip=650
/ltaodataservice/BusRoutes?$skip=700
/ltaodataservice/BusRoutes?$skip=750
/ltaodataservice/BusRoutes?$skip=800
/ltaodataservice/BusRoutes?$skip=850
/ltaodataservice/BusRoutes?$skip=900
/ltaodataservice/BusRoutes?$skip=950
/ltaodataservice/BusRoutes?$skip=1000
/ltaodataservice/BusRoutes?$skip=1050
/ltaodataservice/BusRoutes?$skip=1100
/ltaodataservice/BusRoutes?$skip=1150
/ltaodataservice/BusRoutes?$skip=1200
/ltaodataservice/BusRoutes?$skip=1250
/ltaodataservice/BusRoutes?$skip=13

In [37]:
BusStops = get('BusStops')

/ltaodataservice/BusStops?$skip=0
/ltaodataservice/BusStops?$skip=50
/ltaodataservice/BusStops?$skip=100
/ltaodataservice/BusStops?$skip=150
/ltaodataservice/BusStops?$skip=200
/ltaodataservice/BusStops?$skip=250
/ltaodataservice/BusStops?$skip=300
/ltaodataservice/BusStops?$skip=350
/ltaodataservice/BusStops?$skip=400
/ltaodataservice/BusStops?$skip=450
/ltaodataservice/BusStops?$skip=500
/ltaodataservice/BusStops?$skip=550
/ltaodataservice/BusStops?$skip=600
/ltaodataservice/BusStops?$skip=650
/ltaodataservice/BusStops?$skip=700
/ltaodataservice/BusStops?$skip=750
/ltaodataservice/BusStops?$skip=800
/ltaodataservice/BusStops?$skip=850
/ltaodataservice/BusStops?$skip=900
/ltaodataservice/BusStops?$skip=950
/ltaodataservice/BusStops?$skip=1000
/ltaodataservice/BusStops?$skip=1050
/ltaodataservice/BusStops?$skip=1100
/ltaodataservice/BusStops?$skip=1150
/ltaodataservice/BusStops?$skip=1200
/ltaodataservice/BusStops?$skip=1250
/ltaodataservice/BusStops?$skip=1300
/ltaodataservice/BusStop

In [7]:
import json
json.dumps(BusRoutes)

NameError: name 'BusRoutes' is not defined

In [20]:
import itertools as it

stopsByRoute = [ (k, list(v)) for k, v in it.groupby(BusRoutes, lambda x : (x['ServiceNo'], x['Direction']))]
stopsByRoute = [ (k, sorted(v, key=lambda x:x['StopSequence'])) for k,v in stopsByRoute]
stopsByRoute[0]

((u'10', 1),
 [{u'BusStopCode': u'75009',
   u'Direction': 1,
   u'Distance': 0,
   u'Operator': u'SBST',
   u'SAT_FirstBus': u'0500',
   u'SAT_LastBus': u'2300',
   u'SUN_FirstBus': u'0500',
   u'SUN_LastBus': u'2300',
   u'ServiceNo': u'10',
   u'StopSequence': 1,
   u'WD_FirstBus': u'0500',
   u'WD_LastBus': u'2300'},
  {u'BusStopCode': u'76059',
   u'Direction': 1,
   u'Distance': 0.6,
   u'Operator': u'SBST',
   u'SAT_FirstBus': u'0502',
   u'SAT_LastBus': u'2302',
   u'SUN_FirstBus': u'0502',
   u'SUN_LastBus': u'2302',
   u'ServiceNo': u'10',
   u'StopSequence': 2,
   u'WD_FirstBus': u'0502',
   u'WD_LastBus': u'2302'},
  {u'BusStopCode': u'76069',
   u'Direction': 1,
   u'Distance': 1.1,
   u'Operator': u'SBST',
   u'SAT_FirstBus': u'0504',
   u'SAT_LastBus': u'2304',
   u'SUN_FirstBus': u'0503',
   u'SUN_LastBus': u'2303',
   u'ServiceNo': u'10',
   u'StopSequence': 3,
   u'WD_FirstBus': u'0504',
   u'WD_LastBus': u'2304'},
  {u'BusStopCode': u'96289',
   u'Direction': 1,
   u

In [36]:
# obtain the (prev, next) pairs
def prev_next(bus_stops):
    for a,b in zip(bus_stops, bus_stops[1:]):
        yield a['BusStopCode'], b['BusStopCode']

stop_pairs = [list(prev_next(sbr[1])) for sbr in stopsByRoute]
stop_pairs = [x for y in stop_pairs for x in y ]
stop_pairs = [x for x in stop_pairs if not x[0].startswith('E') and not x[1].startswith('E')]
stop_pairs = list(set(stop_pairs))

In [57]:
import pyproj
svy = pyproj.Proj(init='epsg:3414')

In [ ]:
# pull additional bus stops from One Map
import gzip
import pickle

om_bus_stops = pickle.load(gzip.open('../1mbs/busstops2.pickle.gz'))
om_bus_stops = [r['SearchResults'][1:] for r in om_bus_stops if len(r['SearchResults']) > 1]
om_bus_stops = [x for y in om_bus_stops for x in y]
om_bus_stops = [x for x in om_bus_stops if x['SEARCHVAL'].endswith(' (BUS STOP)')]
for bs in om_bus_stops:
    x,y = bs['X'], bs['Y']
    lng,lat = svy(x,y, inverse=True)
    
    bs['Latitude'] = lat
    bs['Longitude'] = lng
    bs['BusStopCode'] = bs['SEARCHVAL'][0:5]
    

om_bus_stops

In [72]:
combined = om_bus_stops + BusStops

# print [x for x in combined if x['BusStopCode'] == '42885']

combined = [c for c in combined if c['Longitude'] != 0]
stop_lat_lngs = {stop['BusStopCode']: stop for stop in combined}
stop_lat_lngs

[{u'Latitude': 0, u'BusStopCode': u'42885', u'Description': u'Express', u'Longitude': 0, u'RoadName': u'PIE'}]


{u'16079': {u'BusStopCode': u'16079',
  u'Description': u'Bef Pasir Panjang PO',
  u'Latitude': 1.29151166702959,
  u'Longitude': 103.77068722200303,
  u'RoadName': u'Pasir Panjang Rd'},
 u'55349': {u'BusStopCode': u'55349',
  u'Description': u'Thomson Gr',
  u'Latitude': 1.38479144169995,
  u'Longitude': 103.8364930927339,
  u'RoadName': u'Lentor Dr'},
 u'22469': {u'BusStopCode': u'22469',
  u'Description': u'Blk 653B',
  u'Latitude': 1.33649062479414,
  u'Longitude': 103.69631873743737,
  u'RoadName': u'Pioneer Rd Nth'},
 u'24029': {u'BusStopCode': u'24029',
  u'Description': u"Mazak S'pore",
  u'Latitude': 1.3224709639188,
  u'Longitude': 103.67132658072524,
  u'RoadName': u'Jln Ahmad Ibrahim'},
 u'16071': {u'BusStopCode': u'16071',
  u'Description': u'Opp Pasir Panjang PO',
  u'Latitude': 1.29128369274019,
  u'Longitude': 103.77109438530917,
  u'RoadName': u'Pasir Panjang Rd'},
 u'49219': {u'BusStopCode': u'49219',
  u'Description': u'Opp PUB Quarter',
  u'Latitude': 1.438537582459

In [33]:
# http://localhost:8989/route?
#point=1.373846%2C103.880539&point=1.305887%2C103.805008&
#type=json&key=&locale=en-US&vehicle=car&weighting=fastest&elevation=false

In [34]:
import requests

In [78]:
def make_request(pair):
    ll1 = stop_lat_lngs[pair[0]]
    ll2 = stop_lat_lngs[pair[1]]
    
    return ['%s,%s' % (ll1['Latitude'], ll1['Longitude']),
            '%s,%s' % (ll2['Latitude'], ll2['Longitude'])]


def get_route(stop_pair):
    res = requests.get('http://localhost:8989/route',
        params={
            "point": [
                make_request(stop_pair)
            ],
            "type": "json",
            "vehicle": "car",
            "weighting": "fastest",
        })
    
    res = json.loads(res.text)
    
    if not 'paths' in res:
        return None
    
    return polyline.decode(res['paths'][0]['points'])

def has_stop_location(stop_pair):
    return stop_pair[0] in stop_lat_lngs \
        and stop_pair[1] in stop_lat_lngs

pair_routes = [(get_route(stop_pair), stop_pair) for stop_pair in stop_pairs if has_stop_location(stop_pair)]
pair_routes = [x for x in pair_routes if x[0] != None]
pair_routes

[([(1.33562, 103.89832),
   (1.33717, 103.89669),
   (1.33723, 103.89654),
   (1.33729, 103.89644),
   (1.33732, 103.89602),
   (1.33727, 103.89582),
   (1.33682, 103.89539),
   (1.3365, 103.89507)],
  (u'71079', u'71029')),
 ([(1.37522, 103.94643),
   (1.37493, 103.94624),
   (1.3741, 103.94562),
   (1.37378, 103.94536),
   (1.37365, 103.94528),
   (1.3734, 103.94518),
   (1.37303, 103.94503),
   (1.37266, 103.94594)],
  (u'77139', u'77029')),
 ([(1.3391, 103.92794),
   (1.33904, 103.92827),
   (1.33898, 103.92846),
   (1.33866, 103.92943)],
  (u'84611', u'84631')),
 ([(1.31694, 103.94742),
   (1.31701, 103.94766),
   (1.3174, 103.94846),
   (1.31768, 103.94917),
   (1.31837, 103.95152),
   (1.31867, 103.95283),
   (1.31883, 103.95328),
   (1.31918, 103.95409),
   (1.31986, 103.95549),
   (1.31999, 103.95573),
   (1.32007, 103.95581),
   (1.32016, 103.95585),
   (1.32033, 103.95588),
   (1.32046, 103.95586),
   (1.32057, 103.95579),
   (1.32162, 103.95497),
   (1.32194, 103.95473)],
 

In [79]:
import folium

In [88]:
m = folium.Map(location=[1.38, 103.8], zoom_start=12)
m.line(pair_routes[100][0])
m

In [118]:
import math

def swap((x,y)):
    return (y,x)

def heading(ll1, ll2):
    xy1 = svy(*swap(ll1))
    xy2 = svy(*swap(ll2))
    
    return (xy2[0] - xy1[0], xy2[1] - xy1[1])

def dedup(ls):
    return [x for x,y in it.groupby(ls)]

deduped_pair_routes = [
    (dedup(path),stops) for path,stops in pair_routes
]

beginning_headings = [
    (heading(path[0], path[1]), stops[0]) for path,stops in deduped_pair_routes
]
end_headings = [
    (heading(path[-2], path[-1]), stops[1]) for path,stops in deduped_pair_routes
]

headings = beginning_headings + end_headings

In [119]:
stop_headings = {x: list([z[0] for z in y]) for x,y in it.groupby(sorted(headings, key=lambda x:x[1]), key=lambda x:x[1])}

In [129]:
multiple = [(s,x) for s,x in stop_headings.items() if len(x) > 1]

In [130]:
def cosine_distance_to_first(ls):
    def cosine_distance((a,b), (c,d)):
        return (a*c + b*d) / math.sqrt(a*a + b*b) / math.sqrt(c*c + d*d)
    
    return [
        cosine_distance(ls[0], l) for l in ls[1:]
    ]

multiple_headings = [ (s, cosine_distance_to_first(l)) for s,l in multiple]

In [138]:
import numpy as np
l = [ (s, similarities) for s, similarities in multiple_headings if np.min(similarities) < 0.9]
l

[(u'46791', [1.0, -1.0]),
 (u'21611', [-1.0]),
 (u'68171', [1.0, 0.7048395604623108]),
 (u'59619', [-1.0, 0.9998537286244605]),
 (u'63209', [1.0, -1.0]),
 (u'N4569', [-0.9999999999999998]),
 (u'95011', [-1.0]),
 (u'14399', [-0.9999892649043256, -0.9999990718421152]),
 (u'67361', [-1.0000000000000002]),
 (u'55111', [0.8646018221444604]),
 (u'43267', [-0.9999999999999998]),
 (u'43261',
  [1.0000000000000002,
   -0.9998268223754438,
   0.9999258153152194,
   0.9999258153152194,
   0.9999258153152194]),
 (u'11379', [0.9999999999999999, -0.04226410568582924]),
 (u'77281', [0.9999999999999998, -0.999992374858899, -0.999992374858899]),
 (u'07529', [1.0, -1.0]),
 (u'16061', [0.8827442476060643, 0.8827442476060643]),
 (u'16249', [-1.0000000000000002]),
 (u'16241', [-1.0]),
 (u'54429', [-1.0]),
 (u'64519', [-1.0]),
 (u'90079', [-1.0000000000000002]),
 (u'57081', [1.0000000000000002, -1.0000000000000002]),
 (u'48131', [-1.0]),
 (u'83281', [-0.9251256709181315]),
 (u'58331', [1.0, -0.9995251814050

In [139]:
[s[0] for s in l]

[u'46791',
 u'21611',
 u'68171',
 u'59619',
 u'63209',
 u'N4569',
 u'95011',
 u'14399',
 u'67361',
 u'55111',
 u'43267',
 u'43261',
 u'11379',
 u'77281',
 u'07529',
 u'16061',
 u'16249',
 u'16241',
 u'54429',
 u'64519',
 u'90079',
 u'57081',
 u'48131',
 u'83281',
 u'58331',
 u'91079',
 u'62251',
 u'24559',
 u'18151',
 u'59009',
 u'06131',
 u'16121',
 u'93129',
 u'46069',
 u'42039',
 u'52119',
 u'80119',
 u'53029',
 u'95191',
 u'97209',
 u'14419',
 u'46809',
 u'18099',
 u'41149',
 u'47651',
 u'47659',
 u'95141',
 u'63269',
 u'28659',
 u'60251',
 u'43151',
 u'46671',
 u'44699',
 u'44691',
 u'49249',
 u'54071',
 u'31009',
 u'47641',
 u'47649',
 u'11009',
 u'77129',
 u'43231',
 u'43019',
 u'21099',
 u'45401',
 u'27479',
 u'49199',
 u'10349',
 u'99171',
 u'N6709',
 u'47671',
 u'78279',
 u'04121',
 u'27459',
 u'47661',
 u'22609',
 u'22351',
 u'77251',
 u'07571',
 u'28111',
 u'96219',
 u'21079',
 u'64481',
 u'17149',
 u'64361',
 u'17091',
 u'84299',
 u'04339',
 u'18211',
 u'24349',
 u'76241',

In [155]:
# Perform heading annotations

from pymongo import MongoClient

client = MongoClient()
db = client.routing

results = db.busStopHeading.find({
        "heading": {"$exists": True},
        "BusStopCode": {"$in": [s[0] for s in l]},
        "Longitude": {"$ne": 0}
    })
heading_annotations = list(results)
heading_annotations_by_stop = {a['BusStopCode']: a for a in heading_annotations}
heading_annotations

[{u'BusStopCode': u'01109',
  u'Description': u'Queen St Ter',
  u'Latitude': 1.30358577565355,
  u'Longitude': 103.85650372998224,
  u'RoadName': u'Queen St',
  u'_id': ObjectId('57ff351d7d1970313b9d8042'),
  u'heading': [[1.303526946320415, 103.85684698820116],
   [1.3030442735942487, 103.85642856359483]]},
 {u'BusStopCode': u'01229',
  u'Description': u'Bef Sultan Mque',
  u'Latitude': 1.30337552638813,
  u'Longitude': 103.85963089198265,
  u'RoadName': u'Nth Bridge Rd',
  u'_id': ObjectId('57ff351d7d1970313b9d804b'),
  u'heading': [[1.3036771111496797, 103.85989397764207],
   [1.3031998014827813, 103.85923415422442]]},
 {u'BusStopCode': u'04121',
  u'Description': u'SMU',
  u'Latitude': 1.29618544092334,
  u'Longitude': 103.84958029974611,
  u'RoadName': u'Stamford Rd',
  u'_id': ObjectId('57ff351d7d1970313b9d80ab'),
  u'heading': [[1.2961098652358607, 103.84971767663957],
   [1.296554997975691, 103.84937435388566]]},
 {u'BusStopCode': u'04251',
  u'Description': u'Opp High St Ctr'

In [170]:
# Get the final list...
def average_heading(xys):
    return np.mean([math.atan2(*xy) for xy in xys])

calculated = [ (s, average_heading(x)) for s, x in stop_headings.items()]

def infer_heading_from_annotation(a):
    (lat1,lng1), (lat2,lng2) = a['heading']
    (lat,lng) = a['Latitude'], a['Longitude']

    (xy1, xy2) = svy(lng1, lat1), svy(lng2, lat2)

    return math.atan2(xy2[0] - xy1[0], xy2[1] - xy1[1])
     
     
annotated = [ (a['BusStopCode'], infer_heading_from_annotation(a)) for a in heading_annotations ]

all_headings = dict(calculated + annotated)
all_headings

{u'46901': 1.0422956626161561,
 u'16079': 1.8175128124990945,
 u'24509': 0.57294130447079183,
 u'55349': -0.77009578161859915,
 u'65091': 0.41616509784646005,
 u'21509': 2.9280496744139621,
 u'16071': -1.3185044953133365,
 u'21349': -2.8738894078464758,
 u'55341': 2.4282771818768234,
 u'22461': -0.20055460564963046,
 u'20051': -1.5486902565871343,
 u'60061': 2.4583241339604682,
 u'11541': -1.3227505050200288,
 u'27281': -0.74420484480273674,
 u'26379': 2.9891990078918518,
 u'23441': 0.045733428708372187,
 u'77161': 0.62301328921910648,
 u'11381': 2.8637598876857053,
 u'43279': -1.3851425738227352,
 u'92099': -1.9446168269402957,
 u'22469': 2.9376803845121935,
 u'05189': -2.6182731837355124,
 u'11389': -0.29804042562066502,
 u'43271': 1.7179891431703174,
 u'02101': -3.0474835077919171,
 u'92091': 1.1803441067810243,
 u'13021': -1.7128268360164405,
 u'54481': 1.4257378113919732,
 u'18241': 1.4130689879115175,
 u'51069': 3.0129544166879239,
 u'92301': -0.40361937152081201,
 u'58391': 0.94

In [173]:
# Update stops with the latlng inferred from annotation
def update(stop):
    if stop['BusStopCode'] not in heading_annotations_by_stop:
        return stop
    
    a = heading_annotations_by_stop[stop['BusStopCode']]
    
    # clone the dict
    d = dict(stop.items())
     
    # update the lat, lng
    (lat1,lng1), (lat2,lng2) = a['heading']
    (lat,lng) = a['Latitude'], a['Longitude']

    xy1, xy2 = svy(lng1, lat1), svy(lng2, lat2)
    x,y = svy(lng, lat)

    # project x,y onto the line xy1-xy2
    length = math.sqrt((xy2[0] - xy1[0]) * (xy2[0] - xy1[0]) + (xy2[1] - xy1[1]) * (xy2[1] - xy1[1]))
    n = (xy2[0] - xy1[0]) / length, (xy2[1] - xy1[1]) / length

    diff = x-xy1[0], y-xy1[1]

    projected_length = np.dot(diff, n)

    px = xy1[0] + n[0] * projected_length
    py = xy1[1] + n[1] * projected_length
    
    upd_lng, upd_lat = svy(px, py, inverse=True)
    
    d['Latitude'] = upd_lat
    d['Longitude'] = upd_lng
    
    print d
    
    return d
     
projected_bus_stops = [update(stop) for stop in BusStops]


{u'Latitude': 1.3033892018614648, u'BusStopCode': u'01109', u'Description': u'Queen St Ter', u'Longitude': 103.85672757875678, u'RoadName': u'Queen St'}
{u'Latitude': 1.3034488961965103, u'BusStopCode': u'01229', u'Description': u'Bef Sultan Mque', u'Longitude': 103.85957849775474, u'RoadName': u'Nth Bridge Rd'}
{u'Latitude': 1.2962239974934744, u'BusStopCode': u'04121', u'Description': u'SMU', u'Longitude': 103.84962964852822, u'RoadName': u'Stamford Rd'}
{u'Latitude': 1.2897807341128036, u'BusStopCode': u'04251', u'Description': u'Opp High St Ctr', u'Longitude': 103.848908985756, u'RoadName': u'Nth Boat Quay'}
{u'Latitude': 1.2921945175425618, u'BusStopCode': u'04339', u'Description': u'Opp Liang Ct', u'Longitude': 103.84499334797408, u'RoadName': u'River Valley Rd'}
{u'Latitude': 1.2915741533120961, u'BusStopCode': u'06131', u'Description': u"The Boys' Brigade HQ", u'Longitude': 103.83228582921421, u'RoadName': u'Zion Rd'}
{u'Latitude': 1.3082988044371262, u'BusStopCode': u'07331', 

In [177]:
# Update stops with the heading...
def update_heading(stop, headings):
    if stop['BusStopCode'] not in headings:
        print stop
        return stop
    
    d = dict(stop.items())
    d['Heading'] = headings[stop['BusStopCode']]
    
    return d
    
    
stops_with_heading = [update_heading(stop, all_headings) for stop in projected_bus_stops if stop['Longitude'] != 0]

{u'Latitude': 1.28829500000607, u'BusStopCode': u'02011', u'Description': u'Victoria Concert Hall', u'Longitude': 103.85208888899206, u'RoadName': u'Fullerton Rd'}
{u'Latitude': 1.29141338913811, u'BusStopCode': u'02031', u'Description': u"St. Andrew's Cath", u'Longitude': 103.85264858011732, u'RoadName': u"St. Andrew's Rd"}
{u'Latitude': 1.27456, u'BusStopCode': u'03241', u'Description': u'Mapletree Anson', u'Longitude': 103.84537, u'RoadName': u'Anson Rd'}
{u'Latitude': 1.27521793245506, u'BusStopCode': u'03311', u'Description': u'Marina Bay Stn', u'Longitude': 103.85527132634913, u'RoadName': u'Marina St'}
{u'Latitude': 1.27496034569367, u'BusStopCode': u'03319', u'Description': u'Opp Marina Bay Stn', u'Longitude': 103.85529711510442, u'RoadName': u'Marina St'}
{u'Latitude': 1.27621825142233, u'BusStopCode': u'03321', u'Description': u'Bef Marina City Pk', u'Longitude': 103.85921874948889, u'RoadName': u'Marina Pl'}
{u'Latitude': 1.27638333301002, u'BusStopCode': u'03329', u'Descrip

In [179]:
len([s for s in stops_with_heading if 'Heading' not in s])

89

In [181]:
json.dump(stops_with_heading, open('./bus-stops-headings.json', 'w'))